In [3]:
import pandas as pd
import numpy as np 
from sklearn.preprocessing import OneHotEncoder

In [4]:
train = pd.read_json("data/train.json")

In [60]:
train_expanded = train.explode('ingredients')
train_expanded.head()

,id,cuisine,ingredients
0,10259,greek,romaine lettuce
0,10259,greek,black olives
0,10259,greek,grape tomatoes
0,10259,greek,garlic
0,10259,greek,pepper


In [61]:
train.head().explode("ingredients").explode("ingredients").sample(10)

,id,cuisine,ingredients
4,13162,indian,black pepper
4,13162,indian,onions
2,20130,filipino,chicken livers
4,13162,indian,butter
2,20130,filipino,eggs
0,10259,greek,romaine lettuce
1,25693,southern_us,tomatoes
0,10259,greek,garbanzo beans
4,13162,indian,boneless chicken skinless thigh
2,20130,filipino,yellow onion


In [56]:
def get_encoded_ingredients(recipe_df):
    "Take a dataframe with columns 'id' and 'ingredients' and one hot encode the ingredients"
    recipe_ingredient_df = recipe_df.explode('ingredients')
    enc = OneHotEncoder(handle_unknown='ignore')
    enc.fit(recipe_ingredient_df[['ingredients']])
    transformed = enc.transform(recipe_ingredient_df[['ingredients']]).toarray()
    transformed_df = pd.DataFrame(data=transformed, 
                                  columns=enc.categories_[0], 
                                  dtype=bool, 
                                  index=recipe_ingredient_df['id']).reset_index()
    recipe_ohe = transformed_df.groupby('id').any().astype(int).reset_index()
    return recipe_ohe

def ohe_cuisine_recipes(recipe_with_cuisine):
    ohe_ingredients_df = get_encoded_ingredients(recipe_with_cuisine)
    full_recipe_df = ohe_ingredients_df.merge(recipe_with_cuisine[['id','cuisine']], on='id')
    return full_recipe_df


In [58]:
ohe_cuisine_recipes(train.head())

,id,bay leaf,black olives,black pepper,boneless chicken skinless thigh,butter,cayenne pepper,chicken livers,chili powder,cooking oil,...,shallots,soy sauce,thyme,tomatoes,vegetable oil,water,wheat,yellow corn meal,yellow onion,cuisine
0,10259,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,greek
1,13162,1,0,1,1,1,1,0,1,0,...,1,0,0,0,0,1,0,0,0,indian
2,20130,0,0,0,0,1,0,1,0,1,...,0,1,0,0,0,0,0,0,1,filipino
3,22213,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,1,0,0,indian
4,25693,0,0,0,0,0,0,0,0,0,...,0,0,1,1,1,0,0,1,0,southern_us
